In [1]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

In [2]:
#DWPR01P
# conn = cx_Oracle.connect(
#     user="AD21236",
#     password="C#ntury123",
#     dsn="racorap33-scan.corp.intranet/dwpr01p_users",
#     encoding="UTF-8")

# CDW01P
# conn = cx_Oracle.connect(
#     user="AD21236",
#     password="C#ntury123",
#     dsn="RACORAP16-SCAN.IDC1.LEVEL3.COM/CDW01P_USERS",
#     encoding="UTF-8")

# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

# DASP001P
# conn = cx_Oracle.connect(
#     user="AD21236",
#     password="L2022$Um",
#     dsn="racorap27-scan.corp.intranet/dasp001p_prod",
#     encoding="UTF-8")

#ASL01P
# conn = cx_Oracle.connect(
#     user="AD21236",
#     password="C#ntury123",
#     dsn="RACORAP16-SCAN.IDC1.LEVEL3.COM/ASL01P_USERS",
#     encoding="UTF-8")

#DWODS02P - UNIQUAL
# conn = cx_Oracle.connect(
#     user="AD21236",
#     password="C3ntury123",
#     dsn="DWODS02Pdb.qintra.com:1523/DWODS02P",
#     encoding="UTF-8")

#CCDWO1P - SALES
# conn = cx_Oracle.connect(
#     user="AD21236",
#     password="C#ntury123",
#     dsn="racorap33-scan.corp.intranet:1521/ccdw01p_users",
#     encoding="UTF-8")

#CCDW02P - SALES

In [3]:
# qualifications

# select distinct DTN, WIRE_CENTER_ID
# from LQSMO.LU_CIRCUITS_O;


# 10/7/22 - update query to count unique LU where servicetype like XGS
query = """
            WITH QD AS (
            SELECT  
                    O.LUID --count distinct LU id's
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  0 as s
                    ,STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
                    ,UNI_SERVICETYPE
                    ,count(distinct LUID) as cnt
                    ,case when UNI_SERVICETYPE like '%XGS%' then count(distinct LUID) end as servicetype_xgs_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
                    ,UNI_SERVICETYPE

            UNION ALL
            
            SELECT  1
                    ,'ALL'
                    ,'GRAND TOTAL'
                    ,'ALL'
                    ,'ALL'
                    ,count(distinct LUID)
                    ,count(distinct case when UNI_SERVICETYPE like '%XGS%' then LUID end)
                    
            FROM    QD
            
            ORDER BY 
                    s
                    ,STATE

                                """

query2 = """
            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_POLYGONID
                    ,O.UNI_SERVICETYPE
                    ,O.UNI_STATUS
                    ,O.TECH_TYPE
                    ,O.FSAI
                    ,O.OLT_TYPE
                    ,O.ADDRESS
                    ,O.OLT
                    ,O.UNI_ECD
                    --O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  *
                    
            FROM    QD

                                """

# query3 = """

# select *
# FROM    LQSMO.LU_QUAL_O O

# """

df_ora = pd.read_sql(query, con=conn)
df_ora_details = pd.read_sql(query2, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df_ora.rename(columns = {"CNT": "LU_QUAL_O_CNT",
                         "UNI_POLYGONID": "POLYGON_ID"}, inplace = True)
df_ora.sort_values(by = ['S','STATE']).head() # contains state/wc/polygon info

,S,STATE,PRIMARY_WIRE_CENTER_ID,POLYGON_ID,UNI_SERVICETYPE,LU_QUAL_O_CNT,SERVICETYPE_XGS_CNT
0,0,AZ,PHNXAZSO,1179123,BIWF,290,NaN
1,0,AZ,TCSNAZCR,1179641,XGS,176,176.0
2,0,AZ,PHNXAZSO,1176356,BIWF,69,NaN
3,0,AZ,TCSNAZMA,1179690,XGS,449,449.0
4,0,AZ,TCSNAZMA,1179704,XGS,547,547.0


In [5]:
ST_df = pd.read_excel(r'Qualifications ST/Data.xlsx',sheet_name = 'Export')

ST_Details_df = ST_df.copy() # create copy to export

ST_df["Polygon ID - Phase Else Site"] = ST_df["Polygon ID - Phase Else Site"].fillna("None")
ST_df = ST_df.groupby(by = ["State","PRIMARY_WIRE_CENTER_ID","Polygon ID - Phase Else Site"], as_index = False)["# of LUs - Phase Else Site"].sum()
ST_df = ST_df.rename(columns = {'State': 'STATE',
                                'Polygon ID - Phase Else Site': 'POLYGON_ID'})
#ST_df

C:\Users\AD21236\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:
# polygon level info starts here

df_ora_state = df_ora.copy() # use this for state summary with unitype cnt
df_ora = df_ora.groupby(['S','STATE','PRIMARY_WIRE_CENTER_ID','POLYGON_ID'], as_index = False).sum()
Q_df = ST_df.merge(df_ora[["STATE","PRIMARY_WIRE_CENTER_ID", "POLYGON_ID","LU_QUAL_O_CNT"]], how = "outer", on = ["PRIMARY_WIRE_CENTER_ID","STATE","POLYGON_ID"]).fillna(0)
Q_df[["# of LUs - Phase Else Site", "LU_QUAL_O_CNT"]] = Q_df[["# of LUs - Phase Else Site", "LU_QUAL_O_CNT"]].astype(int)
Q_df.loc[Q_df["PRIMARY_WIRE_CENTER_ID"] == "GRAND TOTAL", "# of LUs - Phase Else Site"] = Q_df["# of LUs - Phase Else Site"].sum()

Q_df = Q_df[['STATE','PRIMARY_WIRE_CENTER_ID','POLYGON_ID','# of LUs - Phase Else Site','LU_QUAL_O_CNT']]
Q_poly_df = Q_df.copy() # polygon level summary ready for export

In [7]:
# wc level information start here
Q_df = Q_df.groupby(by = ['STATE','PRIMARY_WIRE_CENTER_ID'], as_index = False).sum() # wc level info ready for export
Q_df['ORDER'] = np.where(Q_df['PRIMARY_WIRE_CENTER_ID'] == 'GRAND TOTAL', 1, 0)
Q_df = Q_df.sort_values(by = 'ORDER')
Q_df = Q_df.drop(['ORDER'], axis = 1)

In [8]:
# state summary begins here

df_ora_state = df_ora_state.fillna(0)
Q_State = df_ora_state.groupby(['STATE'], as_index = False).sum()
Q_State['STATE'] = Q_State['STATE'].replace(['ALL'], 'GRAND TOTAL')
Q_State['ORDER'] = np.where(Q_State['STATE'] == 'GRAND TOTAL', 1, 0)
Q_State = Q_State.sort_values(by = 'ORDER')
Q_State = Q_State.drop(['ORDER','S'], axis = 1)
Q_State

,STATE,LU_QUAL_O_CNT,SERVICETYPE_XGS_CNT
1,AZ,11848,6681.0
2,CO,5573,5573.0
3,FL,13665,13575.0
4,MN,4483,4483.0
5,OR,74,74.0
6,UT,810,810.0
7,WA,1929,1929.0
0,GRAND TOTAL,38382,33125.0


In [9]:
# join lu qual to sitetracker at state level
Q_df_st = Q_df[['STATE','# of LUs - Phase Else Site']].groupby('STATE', as_index = False).sum()
Q_df_st.loc[Q_df_st['STATE'] == 'ALL','STATE'] = 'GRAND TOTAL'
Q_State = Q_State.merge(Q_df_st, how = 'outer', on = 'STATE')
Q_State = Q_State.loc[(Q_State['# of LUs - Phase Else Site']>0) | (Q_State['LU_QUAL_O_CNT']>0),['STATE','# of LUs - Phase Else Site','LU_QUAL_O_CNT','SERVICETYPE_XGS_CNT']]
Q_State

,STATE,# of LUs - Phase Else Site,LU_QUAL_O_CNT,SERVICETYPE_XGS_CNT
0,AZ,7468,11848.0,6681.0
1,CO,6580,5573.0,5573.0
2,FL,14324,13665.0,13575.0
3,MN,4484,4483.0,4483.0
4,OR,76,74.0,74.0
5,UT,1011,810.0,810.0
6,WA,1480,1929.0,1929.0
7,GRAND TOTAL,35423,38382.0,33125.0


In [10]:
import time

TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_LUQ_SITETRACKER_Summary.xlsx"


with pd.ExcelWriter(excelfilename) as writer:
    Q_poly_df.to_excel(writer, sheet_name='PolygonID_Summary', index = False)
    Q_df.to_excel(writer, sheet_name = "WC_Summary", index=False)
    Q_State.to_excel(writer, sheet_name='State_Summary', index = False)
    ST_Details_df.to_excel(writer, sheet_name='ST_Details', index = False)
    df_ora_details.to_excel(writer, sheet_name='LUQUAL_Details', index = False)
    

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()

In [11]:
conn.close()